import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
import pickle
from sklearn.linear_model import LogisticRegression

In [4]:
df = pd.read_csv('avg_by_game_2')
odds = pd.read_csv('nba_game_odds')

In [9]:
df.rename(columns={'Unnamed: 0': 'GameID'}, inplace=True)

In [12]:
df.set_index('GameID', inplace=True)

In [15]:
odds.drop(columns='GameID.1', inplace=True)

In [16]:
odds.set_index('GameID', inplace=True)

In [13]:
df.head()

,Home GP,Home Win Rate,Home Points For,Home FT%,Home ORB,Home DRB,Home TOV,Home STL,Home BLK,Home AST%,...,Away STL,Away BLK,Away AST%,Away eFG%,Away FTr,Away 3PAr,Away Points Against,Home Win By (Vegas),Season,Home Win By
GameID,,,,,,,,,,,,,,,,,,,,,
2011-12-25 00:00:00 Boston @ New York,1,1.0,106.0,0.794118,8.0,23.0,16.0,9.0,11.0,48.599998,...,7.0,5.0,71.800003,0.526,0.408,0.066,106.0,4.5,2011.0,2.0
2011-12-25 00:00:00 Chicago @ Lakers,1,0.0,87.0,0.550000,12.0,30.0,17.0,6.0,8.0,61.099998,...,14.0,4.0,58.299999,0.444,0.157,0.169,87.0,-5.0,2011.0,-1.0
2011-12-25 00:00:00 Clippers @ Golden State,1,0.0,86.0,0.708333,17.0,31.0,16.0,4.0,8.0,53.099998,...,9.0,8.0,59.500000,0.506,0.469,0.284,86.0,-4.5,2011.0,-19.0
2011-12-25 00:00:00 Miami @ Dallas,1,0.0,94.0,0.766667,8.0,23.0,17.0,10.0,0.0,74.199997,...,9.0,6.0,47.400002,0.513,0.462,0.090,94.0,-4.5,2011.0,-11.0
2011-12-25 00:00:00 Orlando @ Oklahoma City,1,1.0,97.0,0.730769,7.0,36.0,16.0,8.0,5.0,61.799999,...,7.0,4.0,60.000000,0.420,0.321,0.346,97.0,7.5,2011.0,8.0


In [17]:
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas)
GameID,,,,,
2011-12-25 00:00:00 Boston @ New York,2011-12-25,2011,New York,Boston,4.5
2011-12-25 00:00:00 Chicago @ Lakers,2011-12-25,2011,Lakers,Chicago,-5.0
2011-12-25 00:00:00 Clippers @ Golden State,2011-12-25,2011,Golden State,Clippers,-4.5
2011-12-25 00:00:00 Miami @ Dallas,2011-12-25,2011,Dallas,Miami,-4.5
2011-12-25 00:00:00 Orlando @ Oklahoma City,2011-12-25,2011,Oklahoma City,Orlando,7.5


In [18]:
df_index = []

for i in range(len(df.index)):
    if df.index[i] not in odds.index:
        df_index.append(df.index[i])

In [22]:
len(df_index) == (len(df) - len(odds))

True

In [23]:
df.drop(index=df_index, inplace=True)

In [25]:
df['Season'] = odds['Season']
df['Home Win By (Vegas)'] = odds['Home Win By (Vegas)']

In [26]:
df.describe()

,Home GP,Home Win Rate,Home Points For,Home FT%,Home ORB,Home DRB,Home TOV,Home STL,Home BLK,Home AST%,...,Away STL,Away BLK,Away AST%,Away eFG%,Away FTr,Away 3PAr,Away Points Against,Home Win By (Vegas),Season,Home Win By
count,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,...,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000,10747.000000
mean,20.147297,0.577667,104.916889,0.759550,10.625557,33.536251,14.047720,7.800882,5.206922,59.393164,...,7.809506,4.761390,57.766951,0.502641,0.268107,0.303521,105.022758,2.630827,2015.493626,2.461803
std,11.456388,0.218615,7.592438,0.042939,1.784050,2.605400,1.626048,1.256312,1.263942,5.229019,...,1.230334,1.057902,4.798240,0.031138,0.043603,0.072985,7.286829,6.684051,2.769182,13.842817
min,1.000000,0.000000,71.000000,0.387097,2.000000,16.000000,6.000000,1.000000,0.000000,32.500000,...,2.000000,0.000000,27.299999,0.328000,0.086000,0.066000,72.000000,-18.000000,2011.000000,-57.000000
25%,10.000000,0.424242,99.552778,0.736275,9.472953,31.866667,13.000000,7.000000,4.387993,56.032048,...,7.070197,4.133333,54.739997,0.482600,0.241428,0.249000,99.972598,-3.000000,2013.000000,-7.000000
50%,20.000000,0.588235,104.833333,0.762295,10.625000,33.571429,13.962963,7.760000,5.071429,59.608327,...,7.733333,4.689655,57.741674,0.501083,0.263526,0.298800,104.666667,3.500000,2015.000000,4.000000
75%,30.000000,0.727273,109.875000,0.785714,11.682373,35.058824,14.923077,8.560488,5.875000,62.793272,...,8.533333,5.333333,60.827619,0.522733,0.290000,0.355341,109.750000,7.500000,2018.000000,11.000000
max,41.000000,1.000000,158.000000,1.000000,26.000000,50.000000,26.000000,18.000000,18.000000,84.400002,...,16.000000,12.000000,83.300003,0.738000,0.685000,0.540167,158.000000,21.500000,2020.000000,61.000000


In [27]:
y = df['Home Win By']
x = df.drop(columns='Home Win By')

In [28]:
linreg = LinearRegression()
rf = RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3, n_estimators=200)
gb = GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5, min_samples_split=3, n_estimators=200, random_state=42)

In [31]:
models = [linreg, rf, gb]

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [30]:
#function to train/test models and prints RMSE
def run_model(x_train, x_test, y_train, y_test, model, scale=False):

    model.fit(x_train, y_train)
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    print('Training RMSE for {} model: {}'.format(model, mean_squared_error(y_train, y_train_pred)**0.5))
    print('Testing RMSE for {} model: {}'.format(model, mean_squared_error(y_test, y_test_pred)**0.5))

In [32]:
for model in models:
    run_model(x_train, x_test, y_train, y_test, model)

Training RMSE for LinearRegression() model: 11.225152165041829
Testing RMSE for LinearRegression() model: 11.409466615048348
Training RMSE for RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3,
                      n_estimators=200) model: 8.869996024592204
Testing RMSE for RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3,
                      n_estimators=200) model: 11.737982256715851
Training RMSE for GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5,
                          min_samples_split=3, n_estimators=200,
                          random_state=42) model: 11.262295262043192
Testing RMSE for GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5,
                          min_samples_split=3, n_estimators=200,
                          random_state=42) model: 11.649645557981255


In [33]:
linreg.fit(x_train, y_train)
rf.fit(x_train, y_train)
gb.fit(x_train, y_train)

linreg_pred = linreg.predict(x_test)
rf_pred = rf.predict(x_test)
gb_pred = gb.predict(x_test)

In [34]:
#function to calculate performance against vegas spread
def beat_vegas(result, vegas, prediction):
    tie = 0
    games = 0
    wins = 0
    win_total = 0
    loss = 0
    loss_total = 0
    
    for i in range(len(result)):
        if ((result[i]>vegas[i]) & (prediction[i]>vegas[i])):
            wins += 1
            games += 1
            win_total += abs(vegas[i]-prediction[i])
            
        elif ((result[i]<vegas[i]) & (prediction[i]<vegas[i])):
            wins += 1
            games += 1
            win_total += abs(vegas[i]-prediction[i])
            
        elif (result[i]==vegas[i]):
            tie += 1
            
        else:
            loss += 1
            games += 1
            loss_total += abs(vegas[i]-prediction[i])
            
    win_rate = (wins/games)*100
    win_distance = win_total/wins
    loss_distance = loss_total/loss
    
    print('Beat Vegas {}% of the time'.format(win_rate))
    print('Vegas RMSE: {}'.format(mean_squared_error(result, vegas)**0.5))
    print('Our RMSE: {}'.format(mean_squared_error(result, prediction)**0.5))
    print('When we beat Vegas, our prediction is {} points away from the Vegas spread.'.format(win_distance))
    print('When we lose to Vegas, our prediction is {} points away from the Vegas spread.'.format(loss_distance))

In [36]:
beat_vegas(y_test, x_test['Home Win By (Vegas)'], linreg_pred)

Beat Vegas 62.358276643990926% of the time
Vegas RMSE: 12.41746477088067
Our RMSE: 11.409466615048348
When we beat Vegas, our prediction is 3.727913282299382 points away from the Vegas spread.
When we lose to Vegas, our prediction is 2.2883684963301643 points away from the Vegas spread.


In [37]:
beat_vegas(y_test, x_test['Home Win By (Vegas)'], rf_pred)

Beat Vegas 61.60241874527589% of the time
Vegas RMSE: 12.41746477088067
Our RMSE: 11.729043588425377
When we beat Vegas, our prediction is 3.213277061924059 points away from the Vegas spread.
When we lose to Vegas, our prediction is 2.1930429274096035 points away from the Vegas spread.


In [39]:
beat_vegas(y_test, x_test['Home Win By (Vegas)'], gb_pred)

Beat Vegas 61.94255479969766% of the time
Vegas RMSE: 12.41746477088067
Our RMSE: 11.649645557981255
When we beat Vegas, our prediction is 3.3676484054293407 points away from the Vegas spread.
When we lose to Vegas, our prediction is 2.26401708279598 points away from the Vegas spread.


In [40]:
linreg2 = LinearRegression()
rf2 = RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3, n_estimators=200)
gb2 = GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5, min_samples_split=3, n_estimators=200, random_state=42)

In [42]:
linreg2.fit(x, y)

LinearRegression()

In [43]:
rf2.fit(x, y)

RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3,
                      n_estimators=200)

In [44]:
gb2.fit(x, y)

GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5,
                          min_samples_split=3, n_estimators=200,
                          random_state=42)

In [45]:
linreg_file = 'linreg_model_2.sav'
pickle.dump(linreg2, open(linreg_file, 'wb'))

rf_file = 'rf_model_2.sav'
pickle.dump(rf2, open(rf_file, 'wb'))

gb_file = 'gb_model_2.sav'
pickle.dump(gb2, open(gb_file, 'wb'))

In [47]:
df.to_csv('avg_by_game_2')